# 单个数据预处理

为构建我们的本地知识库，我们需要对以多种类型存储的本地文档进行处理，读取本地文档并通过前文描述的 Embedding 方法将本地文档的内容转化为词向量来构建向量数据库。接下来以一些实际示例入手，来讲解如何对本地文档进行处理。
## 数据读取
将知识库源数据放置在../../data_base/knowledge_db 目录下

**PDF文档为例**  
可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [1]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

/opt/miniconda3/envs/env_rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


文档加载后储存在 `pdf_pages` 变量中:
- `pdf_pages` 的变量类型为 `List`
- 打印 `pdf_pages` 的长度可以看到 pdf 一共包含多少页

In [2]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 29 页


`pdf_pages` 中的每一元素为一个文档，变量类型为 `langchain_core.documents.base.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [3]:
pdf_page = pdf_pages[20]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'Apache FOP Version 2.6', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'total_pages': 29, 'format': 'PDF 1.4', 'title': '目录', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20230523214533+08'00'", 'page': 20}
------
查看该文档的内容:
7 可靠性设计
容错能力
ZXVMAX-S通过对关键软件资源的定时检测、实时任务监控、存储保护、数据校验、操作日志
信息保存等手段，可有效地防止小软件故障对系统所造成的冲击，提高了软件系统的容错能力
（即软件错误情况下的自愈能力）。
故障监视及处理
ZXVMAX-S具备自动检测与诊断系统软硬件故障的功能，可对故障硬件实施自动隔离、倒换、
重新启动、重新加载等处理。
满足漏洞扫描
为了避免存在安全漏洞使在网设备受外部攻击，需要使系统满足主流安全漏洞扫描工具的要求。
ZXVMAX-S满足Nessus和CD工具安全漏洞扫描的要求。
7.3 数据可靠性
ZXVMAX-S产品在不同分层架构下的数据都可以确保数据可靠和可恢复。

采集层
采集层运行环境部署采用冗余磁盘阵列设计，在服务器损坏、单一硬盘故障等场景下，通过
替换故障硬件后即可工作，数据不会丢失。同时，系统支持数据补充采集，确保系统数据连
续性。

存储共享层
存储共享层的MPP数据库，根据需

In [4]:
# 假设环境中缺少 NLTK 数据包中的 punkt 资源，这是 NLTK 库中用于分词的一个重要组件。解决这个问题的方法是按照报错信息中提到的步骤下载并安装 punkt 资源。
# import nltk
# nltk.download('punkt', quiet=True)

## 数据清洗

我们期望知识库的数据尽量是有序的、优质的、精简的，因此我们要删除低质量的、甚至影响理解的文本数据。  
可以看到上文中读取的pdf文件不仅将一句话按照原文的分行添加了换行符`\n`，也在原本两个符号中间插入了`\n`，我们可以使用正则表达式匹配并删除掉`\n`。

**处理单个page**

In [5]:
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

In [6]:
pdf_page.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        pdf_page.page_content
    )
    
# 2. 批量清理特殊符号
for char in special_chars:
        pdf_page.page_content = pdf_page.page_content.replace(char, '')
    
# 3. 安全删除空格（保留URL等特殊场景）
pdf_page.page_content = space_pattern.sub('', pdf_page.page_content)

pdf_page.page_content

'7可靠性设计容错能力\nZXVMAX-S通过对关键软件资源的定时检测、实时任务监控、存储保护、数据校验、操作日志信息保存等手段，可有效地防止小软件故障对系统所造成的冲击，提高了软件系统的容错能力（即软件错误情况下的自愈能力）。故障监视及处理\nZXVMAX-S具备自动检测与诊断系统软硬件故障的功能，可对故障硬件实施自动隔离、倒换、重新启动、重新加载等处理。满足漏洞扫描为了避免存在安全漏洞使在网设备受外部攻击，需要使系统满足主流安全漏洞扫描工具的要求。\nZXVMAX-S满足Nessus和CD工具安全漏洞扫描的要求。\n7.3\xa0数据可靠性\nZXVMAX-S产品在不同分层架构下的数据都可以确保数据可靠和可恢复。\uf06c采集层采集层运行环境部署采用冗余磁盘阵列设计，在服务器损坏、单一硬盘故障等场景下，通过替换故障硬件后即可工作，数据不会丢失。同时，系统支持数据补充采集，确保系统数据连续性。\uf06c存储共享层存储共享层的MPP数据库，根据需要都设置了多个冗余备份，单一节点的损坏不会影响整系统的运行，也不会导致数据丢失。在替换损坏硬件后，系统能自动进行数据冗余备份的补全和负载均衡。存储共享层的MPP数据库，设计运行在冗余磁盘阵列上，并且是具备双控制器的外置磁阵，在确保性能的同时避免了磁阵数据丢失。\uf06c应用分析层应用分析层操作系统和软件模块都部署在冗余磁盘阵列上，系统软件、配置数据等信息在单一磁盘损坏，主机故障等情形下都不会丢失。\n7.4\xa0组网可靠性\nZXVMAX-S产品根据需要，可以通过交换机冗余备份，各主机网络的以太网隧道冗余备份，实现对业务和设备的保护。在单一交换机故障时，备用交换机会及时启用，避免业务中断。在单一的网络传输故障时，备用的传输链路仍然生效，也能避免业务中断。\nSJ-20220623151803-017|2023-03-30（R1.0）\n17'

## 文档分割

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 `chunk_size` (块大小)和 `chunk_overlap` (块与块之间的重叠大小)进行分割。

![image.png](../../assets/rag8.png)

* chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量

* chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

**Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小**

- RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。
- CharacterTextSplitter(): 按字符来分割文本。
- MarkdownHeaderTextSplitter(): 基于指定的标题来分割markdown 文件。
- TokenTextSplitter(): 按token来分割文本。
- SentenceTransformersTokenTextSplitter(): 按token来分割文本
- Language(): 用于 CPP、Python、Ruby、Markdown 等。
- NLTKTextSplitter(): 使用 NLTK（自然语言工具包）按句子分割文本。
- SpacyTextSplitter(): 使用 Spacy按句子的切割文本。

In [7]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [9]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(pdf_page.page_content[0:1000])

In [10]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：55


In [11]:
split_docs[50].page_content

'10\xa0遵循标准与要求\n本章包含如下主题：\n\uf06c 运维标准\n23\n\uf06c 安全标准\n23\n10.1\xa0运维标准\nZXVMAX-S产品符合TMF以下标准：GB923，GB921，GB917，GB922，TR148，TR149\n10.2\xa0安全标准\nZXVMAX-S产品符合以下安全标准：\n\uf06c\nCIS 安全加固标准\n\uf06c\nISO/IEC 15408：信息技术安全评估通用准则（CC，Common Criteria，对应的国标是GB/\nT 18336）\n\uf06c\nGB/T 18336：信息技术 安全技术 信息技术安全性评估准则\n\uf06c\nUL 60950-1：ed.2/CSA-C22.2 No.60950-1-07:ed.2\nSJ-20220623151803-017 | 2023-03-30（R1.0）\n23'

In [12]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：18934


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在本章，我们仅简单根据 chunk_size 对文档进行分割。对于有兴趣进一步探索的读者，欢迎阅读我们第三部分的项目示例来参考已有的项目是如何进行文档分割的。

# 批量数据预处理

批量处理文件夹中所有文件

In [13]:
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")


# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts


# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )
    
    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')
    
    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)

['../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（端到端业务）.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）故障管理概述.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（语音业务）.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.20.80.02）告警处理.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（数据业务）.pdf', '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf']


In [14]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [16]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
# text_splitter.split_text(text.page_content[0:1000])

In [17]:
split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：891


In [18]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：319269


In [19]:
split_docs[300].page_content

'7.2004000200284SGS接口ServiceIndicator合规率(小时)...................269\n7.2014000200285SGS接口NewLAC合规率(小时).............................269\n7.2024000200286SGS接口OldLAC合规率(小时).............................270\n7.2034000200287SGS接口TAC合规率(小时).................................270\n7.2044000200288SGS接口CellID合规率(小时).............................270\n7.2054000200289HTTP接口AppType完整率(小时)............................271\n7.2064000200290HTTP接口AppSub-type完整率(小时)........................271'